In [14]:
import os
import shutil

import mlflow
from random import random, randint
from mlflow import log_metric, log_param, log_artifact, log_artifacts

mlflow.set_experiment("selection")

<Experiment: artifact_location='file:///files0/extended/bohm747/Development/soda/soda-opt/experiments/selection/mlruns/1', experiment_id='1', lifecycle_stage='active', name='selection', tags={}>

In [2]:
mlflow.start_run()

<ActiveRun: >

In [15]:

cwd=os.getcwd()

output_root=os.path.join(cwd,'output')
bambu_root=os.path.join(output_root,'bambu')

try:
    shutil.rmtree(bambu_root)
except OSError as e:
    print("Error: %s : %s" % (bambu_root, e.strerror))

output_paths= [output_root, bambu_root]

for d in output_paths:
    if not os.path.exists(d):
        os.makedirs(d)

In [4]:
%%bash -s {output_root}

ls $1

00_source.mlir
01_searched.mlir
02_outlined.mlir
03_isolated.mlir
04_intermediate.mlir
05_optimized.mlir
06_llvm.ll
07_model.ll
bambu
bambu-exec-log
kernel_kernel_interface.xml
kernel_kernel_test.xml
main_kernel_interface.xml
main_kernel_test.xml


In [5]:
%%bash
ODIR=output

cp kernel.mlir ${ODIR}/00_source.mlir

# Mark region with soda.launch ############################################
soda-opt \
  --convert-affine-for-to-soda \
  ${ODIR}/00_source.mlir \
  -o ${ODIR}/01_searched.mlir

# Outline region inside soda.launch ############################################
soda-opt \
  --soda-outline-bambu-code \
  --soda-extract-arguments-to-xml=using-bare-ptr \
  ${ODIR}/01_searched.mlir \
  -o ${ODIR}/02_outlined.mlir

mv main_kernel_test.xml ${ODIR}/bambu/main_kernel_test.xml
mv main_kernel_interface.xml ${ODIR}/bambu/main_kernel_interface.xml

# Isolate the outlined region in a separate file ###############################
soda-opt \
  -soda-generate-bambu-accelcode \
  ${ODIR}/02_outlined.mlir \
  -o ${ODIR}/03_isolated.mlir


# Optimize the isolated code
soda-opt \
    --soda-opt-pipeline-for-bambu="use-bare-ptr-memref-call-conv" \
    ${ODIR}/03_isolated.mlir \
    -o ${ODIR}/05_optimized.mlir \
    -print-ir-before-all 2>&1 | cat > ${ODIR}/04_intermediate.mlir

# Translate llvm.mlir to model.ll ##############################################
mlir-translate \
  --mlir-to-llvmir \
  ${ODIR}/05_optimized.mlir \
  -o ${ODIR}/06_llvm.ll

# Remove debug info to make output file smaller ################################
opt-10 \
  --strip-debug -O2 \
  ${ODIR}/06_llvm.ll \
  -S -o ${ODIR}/07_model.ll

cp ${ODIR}/07_model.ll ${ODIR}/bambu/input.ll

In [6]:
%%bash

ODIR=output
mkdir -p ${ODIR}/bambu
cp ${ODIR}/07_model.ll ${ODIR}/bambu/input.ll

pushd ${ODIR}/bambu


docker run -u $(id -u):$(id -g) -v $(pwd):/working_dir --rm soda/soda \
bambu -v3 --print-dot \
  -lm --soft-float \
  --compiler=I386_CLANG10  \
  -O2 \
  --device=nangate45 \
  --clock-period=5 --no-iob \
  --experimental-setup=BAMBU-BALANCED-MP \
  --channels-number=2 \
  --memory-allocation-policy=ALL_BRAM \
  --disable-function-proxy \
  --generate-tb=main_kernel_test.xml \
  --simulate --simulator=VERILATOR \
  --top-fname=main_kernel \
  input.ll 2>&1 | tee bambu-exec-log

popd

/files0/extended/bohm747/Development/soda/soda-opt/experiments/selection/output/bambu /files0/extended/bohm747/Development/soda/soda-opt/experiments/selection
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG10 -O2 --device=nangate45 --clock-period=5 --no-iob --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_test.xml --simulate --simulator=VERILATOR --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
                         High-Level Synthesis Tool

 

In [7]:
result = 0 
completed=False
metrics = {}

with open(os.path.join(bambu_root, 'bambu-exec-log')) as f:
  for l in f.readlines():
    if ('SUCCESS' in l):
      completed=True

    if ('Total cycles' in l):
      result = int(l.split(sep=':')[1].split()[0])

if(completed):
  mlflow.set_tag("LOG_STATUS", "SUCCESS")
else:
  mlflow.set_tag("LOG_STATUS", "FAIL")


metrics={'cycles': result}

In [8]:

if (False):
    sodaopt_outlining_options=[
    ]
    sodaopt_baseline_options=[
        """
        """
    ]
    sodaopt_optimization_options=[
        """
            -convert-linalg-to-affine-loops \
            -affine-data-copy-generate="generate-dma=false fast-mem-space=0" \
            -erase-buffer-deallocation \
            --promote-buffers-to-stack="max-rank-of-allocated-memref=4 max-alloc-size-in-bytes=4096" \
            -affine-loop-unroll="unroll-full" \
            -affine-loop-unroll="unroll-full" \
            -affine-loop-unroll="unroll-full" \
            -affine-scalrep -cse \
            -lower-affine -convert-scf-to-cf -convert-memref-to-llvm \
            --convert-math-to-llvm --convert-math-to-libm \
            -arith-expand   \
            -memref-expand  \
            --convert-arith-to-llvm \
            -convert-std-to-llvm  \
            -reconcile-unrealized-casts
    """
    ]

    'soda-opt input.mlir -o '+ os.path.join(output_root, '00_source.mlir')

In [9]:

sodaopt_string="""
soda-opt \
    --soda-opt-pipeline-for-bambu="use-bare-ptr-memref-call-conv" \
    ${ODIR}/03_isolated.mlir \
    -o ${ODIR}/05_optimized.mlir \
    -print-ir-before-all 2>&1 | cat > ${ODIR}/04_intermediate.mlir
"""

bambu_string="""
docker run -u $(id -u):$(id -g) -v $(pwd):/working_dir --rm soda/soda \
bambu -v3 --print-dot \
  -lm --soft-float \
  --compiler=I386_CLANG10  \
  -O2 \
  --device=nangate45 \
  --clock-period=5 --no-iob \
  --experimental-setup=BAMBU-BALANCED-MP \
  --channels-number=2 \
  --memory-allocation-policy=ALL_BRAM \
  --disable-function-proxy \
  --generate-tb=main_kernel_test.xml \
  --simulate --simulator=VERILATOR \
  --top-fname=main_kernel \
  input.ll 2>&1 | tee bambu-exec-log
"""

artifacts = [
    os.path.join(output_root, '00_source.mlir'),
    os.path.join(output_root, '01_searched.mlir'),
    os.path.join(output_root, '02_outlined.mlir'),
    os.path.join(output_root, '03_isolated.mlir'),
    os.path.join(output_root, '04_intermediate.mlir'),
    os.path.join(output_root, '05_optimized.mlir'),
    os.path.join(output_root, '06_llvm.ll'),
    os.path.join(output_root, '07_model.ll'),
    os.path.join(bambu_root,  'bambu-exec-log'),
    os.path.join(bambu_root,  'main_kernel.v'),
    os.path.join(bambu_root,  'HLS_output/dot/main_kernel/HLS_STGraph.dot')
]

In [10]:

log_param("sodaopt_string", sodaopt_string)
log_param("bambu_string", bambu_string)
for artifact in artifacts:
  log_artifact(artifact)

# bambu has a symlink, so this is not possible
# log_artifacts(output_root)

if (completed):
  for k,v in metrics.items():
    log_metric(k, v)

In [11]:
mlflow.end_run()